# blog 크롤링

## 크롤링 조건 설정

In [43]:
# 크롤링할 페이지 설정
number_page = 4
# 날짜 설정 : "설정 날짜" 포함, 이후 날짜 데이터 출력됨.
start_date = '2022-01-01'

In [44]:
from bs4 import BeautifulSoup as bs
import time
import random
from selenium import webdriver
import pandas as pd
from datetime import datetime

## 크롤링할 페이지 접속

In [45]:
# 페이지 접속
driver = webdriver.Chrome('chromedriver')
url = 'https://www.gyeonggido-korea.com/'
# webdriver 로드
driver.get(url)
time.sleep(3)
driver.implicitly_wait(2)

# 창 최대화
driver.maximize_window()
time.sleep(1)

In [46]:
# 페이지가 스프에 담김 확인
soup = bs(driver.page_source, 'lxml')
print("타이틀 : ", soup.title.text)

타이틀 :  Gyeonggido-Korea (경기도)


# 해당 페이지 크롤링 함수

In [47]:
def page_crawling(url, number_page):
    # 페이지로 이동
    driver.get(url)
    time.sleep(1)
    driver.implicitly_wait(3)

    # 페이지 이동 버튼 xpath 수집
    page_link_xpaths = driver.find_elements_by_xpath('//*[@id="HTML4"]/div[3]/a')
    print("페이지 이동버튼 개수 : ", len(page_link_xpaths))

    # 각 페이지에서 게시글 크롤링
    page_title_list = []
    page_date_list = []
    post_link_list = []

    for i in range(len(page_link_xpaths[:number_page])):
        page_button = driver.find_element_by_xpath(f'//*[@id="HTML4"]/div[3]/a[{i+1}]')
        page_button.click()
        driver.implicitly_wait(3)

        time.sleep( random.uniform(1, 2) ) # 크롤링 차단 방지
        page_soup = bs(driver.page_source, 'lxml')

        # 게시글 제목 수집
        titles = page_soup.find_all('h3', 'item-title')
        
        for title in titles:
            temp = title.find('a').text
            page_title_list.append(temp)

        # 게시글 날짜 수집
        dates = page_soup.find_all('div', 'meta-items')

        for date in dates:
            temp = date.find('a', 'meta-item meta-item-date').find('span').text
            page_date_list.append(temp)

        # 게시글 링크 수집
        for link in titles:
            temp = link.find('a').attrs['href']
            post_link_list.append(temp)

    # 수집한 데이터 확인
    print("수집한 제목 수 : ", len(page_title_list))
    print("수집한 날짜 수 : ", len(page_date_list))
    print("수집한 링크 수 : ", len(post_link_list))

    # 날짜 형식 변환
    mod_page_date_list = []
    for raw_date in page_date_list:
        temp = raw_date.replace(',', '')
        temp_list = temp.split(' ')
        year = temp_list[2]
        day = temp_list[1]
        dic = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06', 'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}
        month = dic[temp_list[0]]
        date = f'{year}-{month}-{day}'
        mod_page_date_list.append(date)

    print("수정된 날짜 개수 : ", len(mod_page_date_list))

    # 조회수, 좋아요 수, 공유수, 댓글 수 기본 리스트 생성('0'값으로 채운 리스트)
    number_view_list = []
    number_like_list = []
    number_share_list = []
    number_comment_list = []

    for i in range(len(mod_page_date_list)):
        number_view_list.append(0)
    for i in range(len(mod_page_date_list)):
        number_like_list.append(0)
    for i in range(len(mod_page_date_list)):
        number_share_list.append(0)
    for i in range(len(mod_page_date_list)):
        number_comment_list.append(0)

    print("조회수 리스트 원소 개수 : ", len(number_view_list))
    print("좋아요 리스트 원소 개수 : ", len(number_like_list))
    print("공유 리스트 원소 개수 : ", len(number_share_list))
    print("댓글 리스트 원소 개수 : ", len(number_comment_list))

    return mod_page_date_list, page_title_list,  number_like_list, number_share_list, number_comment_list, post_link_list

## 각 항목 url

In [48]:
url_list = {'about':'https://www.gyeonggido-korea.com/search/label/about', 
            'business':'https://www.gyeonggido-korea.com/search/label/business', 
            'policy':'https://www.gyeonggido-korea.com/search/label/policy', 
            'visiting':'https://www.gyeonggido-korea.com/search/label/visiting', 
            'living':'https://www.gyeonggido-korea.com/search/label/living', 
            'news':'https://www.gyeonggido-korea.com/search/label/news'}

# 각 항목 크롤링

## About 항목 크롤링

In [49]:
url = url_list['about']
about_page_date_list, about_page_title_list,  about_number_like_list, about_number_share_list, about_number_comment_list, about_post_link_list = page_crawling(url, number_page)

<ipython-input-47-d1ff70f248a5>:8: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  page_link_xpaths = driver.find_elements_by_xpath('//*[@id="HTML4"]/div[3]/a')
<ipython-input-47-d1ff70f248a5>:17: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  page_button = driver.find_element_by_xpath(f'//*[@id="HTML4"]/div[3]/a[{i+1}]')


페이지 이동버튼 개수 :  10
수집한 제목 수 :  28
수집한 날짜 수 :  28
수집한 링크 수 :  28
수정된 날짜 개수 :  28
조회수 리스트 원소 개수 :  28
좋아요 리스트 원소 개수 :  28
공유 리스트 원소 개수 :  28
댓글 리스트 원소 개수 :  28


## Business 페이지 크롤링

In [50]:
url = url_list['business']
business_page_date_list, business_page_title_list,  business_number_like_list, business_number_share_list, business_number_comment_list, business_post_link_list = page_crawling(url, number_page)

<ipython-input-47-d1ff70f248a5>:8: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  page_link_xpaths = driver.find_elements_by_xpath('//*[@id="HTML4"]/div[3]/a')
<ipython-input-47-d1ff70f248a5>:17: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  page_button = driver.find_element_by_xpath(f'//*[@id="HTML4"]/div[3]/a[{i+1}]')


페이지 이동버튼 개수 :  9
수집한 제목 수 :  28
수집한 날짜 수 :  28
수집한 링크 수 :  28
수정된 날짜 개수 :  28
조회수 리스트 원소 개수 :  28
좋아요 리스트 원소 개수 :  28
공유 리스트 원소 개수 :  28
댓글 리스트 원소 개수 :  28


## Policy 페이지 크롤링

In [51]:
url = url_list['policy']
policy_page_date_list, policy_page_title_list,  policy_number_like_list, policy_number_share_list, policy_number_comment_list, policy_post_link_list = page_crawling(url, number_page)

<ipython-input-47-d1ff70f248a5>:8: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  page_link_xpaths = driver.find_elements_by_xpath('//*[@id="HTML4"]/div[3]/a')
<ipython-input-47-d1ff70f248a5>:17: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  page_button = driver.find_element_by_xpath(f'//*[@id="HTML4"]/div[3]/a[{i+1}]')


페이지 이동버튼 개수 :  9
수집한 제목 수 :  28
수집한 날짜 수 :  28
수집한 링크 수 :  28
수정된 날짜 개수 :  28
조회수 리스트 원소 개수 :  28
좋아요 리스트 원소 개수 :  28
공유 리스트 원소 개수 :  28
댓글 리스트 원소 개수 :  28


## Visiting 항목 크롤링

In [52]:
url = url_list['visiting']
visiting_page_date_list, visiting_page_title_list,  visiting_number_like_list, visiting_number_share_list, visiting_number_comment_list, visiting_post_link_list = page_crawling(url, number_page)

<ipython-input-47-d1ff70f248a5>:8: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  page_link_xpaths = driver.find_elements_by_xpath('//*[@id="HTML4"]/div[3]/a')
<ipython-input-47-d1ff70f248a5>:17: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  page_button = driver.find_element_by_xpath(f'//*[@id="HTML4"]/div[3]/a[{i+1}]')


페이지 이동버튼 개수 :  13
수집한 제목 수 :  28
수집한 날짜 수 :  28
수집한 링크 수 :  28
수정된 날짜 개수 :  28
조회수 리스트 원소 개수 :  28
좋아요 리스트 원소 개수 :  28
공유 리스트 원소 개수 :  28
댓글 리스트 원소 개수 :  28


## Living 항목 크롤링

In [53]:
url = url_list['living']
living_page_date_list, living_page_title_list,  living_number_like_list, living_number_share_list, living_number_comment_list, living_post_link_list = page_crawling(url, number_page)

<ipython-input-47-d1ff70f248a5>:8: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  page_link_xpaths = driver.find_elements_by_xpath('//*[@id="HTML4"]/div[3]/a')
<ipython-input-47-d1ff70f248a5>:17: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  page_button = driver.find_element_by_xpath(f'//*[@id="HTML4"]/div[3]/a[{i+1}]')


페이지 이동버튼 개수 :  10
수집한 제목 수 :  28
수집한 날짜 수 :  28
수집한 링크 수 :  28
수정된 날짜 개수 :  28
조회수 리스트 원소 개수 :  28
좋아요 리스트 원소 개수 :  28
공유 리스트 원소 개수 :  28
댓글 리스트 원소 개수 :  28


## News&Event 항목 크롤링

In [54]:
url = url_list['news']
news_page_date_list, news_page_title_list,  news_number_like_list, news_number_share_list, news_number_comment_list, news_post_link_list = page_crawling(url, number_page)

<ipython-input-47-d1ff70f248a5>:8: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  page_link_xpaths = driver.find_elements_by_xpath('//*[@id="HTML4"]/div[3]/a')
<ipython-input-47-d1ff70f248a5>:17: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  page_button = driver.find_element_by_xpath(f'//*[@id="HTML4"]/div[3]/a[{i+1}]')


페이지 이동버튼 개수 :  6
수집한 제목 수 :  28
수집한 날짜 수 :  28
수집한 링크 수 :  28
수정된 날짜 개수 :  28
조회수 리스트 원소 개수 :  28
좋아요 리스트 원소 개수 :  28
공유 리스트 원소 개수 :  28
댓글 리스트 원소 개수 :  28


## 웹드라이버 종료

In [55]:
# chromedriver 종료
driver.quit()

# 파일로 출력

## 수집한 데이터 병합

In [56]:
# 날짜 병합
date_list = about_page_date_list + business_page_date_list + policy_page_date_list + visiting_page_date_list + living_page_date_list + news_page_date_list
# 제목 병합
title_list = about_page_title_list + business_page_title_list + policy_page_title_list + visiting_page_title_list + living_page_title_list + news_page_title_list
# 조회수 기본 리스트 생성('0' 값으로 채운 리스트)
number_view_list = []
for i in range(len(date_list)):
    number_view_list.append(0)
# 좋아요 수 병합
number_like_list = about_number_like_list * 6
# 공유 수 병합
number_share_list = about_number_share_list * 6
# 댓글 수 병합
number_comment_list = about_number_comment_list * 6

In [57]:
# 링크 병합
post_link_list = about_post_link_list + business_post_link_list + policy_post_link_list + visiting_post_link_list + living_post_link_list + news_post_link_list
len(post_link_list)

168

In [58]:
print("수집한 날짜 수 : ", len(date_list))
print("수집한 제목 수 : ", len(title_list))
print("수집한 조회수 리스트 원소 수 : ", len(number_view_list))
print("수집한 좋아요 수 리스트 원소 수 : ", len(number_like_list))
print("수집한 공유 수 리스트 원소 수 : ", len(number_share_list))
print("수집한 댓글 수 리스트 원소 수 : ", len(number_comment_list))

수집한 날짜 수 :  168
수집한 제목 수 :  168
수집한 조회수 리스트 원소 수 :  168
수집한 좋아요 수 리스트 원소 수 :  168
수집한 공유 수 리스트 원소 수 :  168
수집한 댓글 수 리스트 원소 수 :  168


## 데이터 셋 생성

In [59]:
# 데이터 프레임 생성
df = {'날짜':date_list, '제목':title_list, '조회수':number_view_list, '좋아요_수':number_like_list, '공유수':number_share_list, '댓글수':number_comment_list, '링크':post_link_list}
data = pd.DataFrame(df)
data

,날짜,제목,조회수,좋아요_수,공유수,댓글수,링크
0,2022-03-28,Gyeonggi's Public Delivery App for Small Busin...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
1,2022-03-25,Gyeonggi Launches Digital Transformation (DX) ...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
2,2022-03-23,Gyeonggi Builds and Operates Korea’s First Fir...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
3,2022-03-21,"A Hot Place in Gyeonggi-do, Flower Mart, Launc...",0,0,0,0,https://www.gyeonggido-korea.com/2022/03/a-hot...
4,2022-03-19,Gyeonggi Provides QR Codes and Foreign Languag...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
...,...,...,...,...,...,...,...
163,2020-09-25,Summary of DMZ Forum2020: A powerful message s...,0,0,0,0,https://www.gyeonggido-korea.com/2020/09/summa...
164,2020-09-15,Korea’s Symbol of Peace: DMZ – 2020 DMZ Forum,0,0,0,0,https://www.gyeonggido-korea.com/2020/09/Korea...
165,2020-08-20,The New Normal for Migrants in Gyeonggi-do | P...,0,0,0,0,https://www.gyeonggido-korea.com/2020/08/the-n...
166,2020-06-08,Gyeonggi Province in the Foreign Press (during...,0,0,0,0,https://www.gyeonggido-korea.com/2020/06/gyeon...


# 중복 샘플 제거

In [60]:
dupl_data = data.drop_duplicates(['날짜']).reset_index(drop=True)
dupl_data

,날짜,제목,조회수,좋아요_수,공유수,댓글수,링크
0,2022-03-28,Gyeonggi's Public Delivery App for Small Busin...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
1,2022-03-25,Gyeonggi Launches Digital Transformation (DX) ...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
2,2022-03-23,Gyeonggi Builds and Operates Korea’s First Fir...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
3,2022-03-21,"A Hot Place in Gyeonggi-do, Flower Mart, Launc...",0,0,0,0,https://www.gyeonggido-korea.com/2022/03/a-hot...
4,2022-03-19,Gyeonggi Provides QR Codes and Foreign Languag...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
...,...,...,...,...,...,...,...
126,2020-09-25,Summary of DMZ Forum2020: A powerful message s...,0,0,0,0,https://www.gyeonggido-korea.com/2020/09/summa...
127,2020-09-15,Korea’s Symbol of Peace: DMZ – 2020 DMZ Forum,0,0,0,0,https://www.gyeonggido-korea.com/2020/09/Korea...
128,2020-08-20,The New Normal for Migrants in Gyeonggi-do | P...,0,0,0,0,https://www.gyeonggido-korea.com/2020/08/the-n...
129,2020-06-08,Gyeonggi Province in the Foreign Press (during...,0,0,0,0,https://www.gyeonggido-korea.com/2020/06/gyeon...


## 데이터 셋. 날짜로 슬라이싱

In [61]:
# 데이터 셋 날짜로 슬라이싱
slicing_data = dupl_data[dupl_data['날짜'] >= start_date].reset_index(drop=True)
slicing_data

,날짜,제목,조회수,좋아요_수,공유수,댓글수,링크
0,2022-03-28,Gyeonggi's Public Delivery App for Small Busin...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
1,2022-03-25,Gyeonggi Launches Digital Transformation (DX) ...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
2,2022-03-23,Gyeonggi Builds and Operates Korea’s First Fir...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
3,2022-03-21,"A Hot Place in Gyeonggi-do, Flower Mart, Launc...",0,0,0,0,https://www.gyeonggido-korea.com/2022/03/a-hot...
4,2022-03-19,Gyeonggi Provides QR Codes and Foreign Languag...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
5,2022-03-17,Gyeonggi Provides Multicultural Newspaper Subs...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
6,2022-03-16,2022 Gyeonggi Foreign Social Media Reporters R...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/2022-...
7,2022-03-15,Gyeonggi Releases 1.8 Million Data Autonomous ...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
8,2022-03-12,Gyeonggi Province’s Namyangju City launches mu...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
9,2022-03-10,Gyeonggi Province improves foreign laborer she...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...


In [62]:
final_data = slicing_data.sort_values(by='날짜')
final_data

,날짜,제목,조회수,좋아요_수,공유수,댓글수,링크
32,2022-01-05,Gyeonggi Startup Platform recognized as South ...,0,0,0,0,https://www.gyeonggido-korea.com/2022/01/gyeon...
31,2022-01-06,A Collection of K-Culture Published in Collabo...,0,0,0,0,https://www.gyeonggido-korea.com/2022/01/a-col...
27,2022-01-07,"5 places to travel in Siheung, Gyeonggi-do Thr...",0,0,0,0,https://www.gyeonggido-korea.com/2022/01/5-pla...
28,2022-01-13,Best Museums in Gyeonggi Province to Experienc...,0,0,0,0,https://www.gyeonggido-korea.com/2022/01/best-...
30,2022-01-15,International Express Mail Service from Korea?...,0,0,0,0,https://www.gyeonggido-korea.com/2022/01/inter...
29,2022-01-17,Gyeonggi Province Institute of Health & Enviro...,0,0,0,0,https://www.gyeonggido-korea.com/2022/01/gyeon...
25,2022-01-20,Experience the electronics industry with the S...,0,0,0,0,https://www.gyeonggido-korea.com/2022/01/exper...
24,2022-01-24,Gyeonggi Province leads vitalization of health...,0,0,0,0,https://www.gyeonggido-korea.com/2022/01/gyeon...
23,2022-01-26,Suwon in the world: Places to visit for living...,0,0,0,0,https://www.gyeonggido-korea.com/2022/01/suwon...
26,2022-01-27,"Gyeonggi Province Big Data Committee convenes,...",0,0,0,0,https://www.gyeonggido-korea.com/2022/01/gyeon...


## 엑셀 파일로 출력

In [63]:
# 파일로 출력
file_name = str(datetime.today().strftime("%Y%m%d-%H%M%S"))
file_name = 'blog_' + file_name + '.xlsx'
path = './crawling_data/blog/'
final_data.to_excel(path+file_name, index=False, encoding='utf-8')

print(f"{file_name} 파일 생성 완료")

blog_20220329-170823.xlsx 파일 생성 완료


# 코드 마지막